In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from imp import reload
import torch.nn as nn

import vnn2
reload(vnn2)
import local2d

In [104]:
in_channels = 4
out_channels = 4
h_in = 24
w_in = 24
kernel_size = 5

lc = local2d.Local2d(in_channels, out_channels, kernel_size, h_in, w_in, stride=1, padding=3, bias=True)

In [105]:
x = torch.randn(128, in_channels, h_in, w_in)
out = lc(x)

In [106]:
loss = out.sum()

In [107]:
loss.backward()

In [108]:
g1 = lc.weight.grad.clone()

In [111]:
W = lc.weight.detach().clone()
W.grad = None
W.requires_grad = True
out = local2d.lc_forward(x.clone(), W, stride=1, padding=3)

In [112]:
loss = out.sum()

In [113]:
loss.backward()

In [114]:
g2 = W.grad.clone()

In [117]:
(g1-g2).abs().max()

tensor(1.1444e-05)

In [2]:
reload(vnn2)
def make_cifar_vec_lc(mono=False):
    model = nn.Sequential(
        vnn2.VecLocal2d(10, 30, 128, 5, h_in=32, w_in=32, stride=1, padding=2, first_layer=True, mono=mono),
        vnn2.ctReLU(10, 128, 32, 32),
        vnn2.AvgPool2d(2),
        vnn2.VecLocal2d(10, 128, 128, 5, h_in=16, w_in=16, stride=1, padding=2, mono=mono),
        vnn2.ctReLU(10, 128, 16, 16),
        vnn2.AvgPool2d(2),
        vnn2.VecLocal2d(10, 128, 128, 2, h_in=8, w_in=8, stride=2, padding=0, mono=mono),
        vnn2.ctReLU(10, 128, 4, 4),
        vnn2.Flatten(),
        vnn2.Linear(10, 2048, 1024, mono=mono),
        vnn2.tReLU(10, 1024),
        vnn2.Linear(10, 1024, 1, mono=mono))
    return model

model = make_cifar_vec_lc(mono=False)



In [3]:
model

Sequential(
  (0): VecLocal2d()
  (1): ctReLU()
  (2): AvgPool2d(
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (3): VecLocal2d()
  (4): ctReLU()
  (5): AvgPool2d(
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (6): VecLocal2d()
  (7): ctReLU()
  (8): Flatten()
  (9): Linear()
  (10): tReLU()
  (11): Linear()
)

In [4]:
input = torch.randn(1, 10, 30, 32, 32)
labels = torch.randint(0, 10, (1,))

In [5]:
output = model(input)[..., 0]
vnn2.set_model_grads(model, output, labels, learning_rule="bp", reduction="mean")
g1 = [param.grad for (name, param) in model.named_parameters() if name[-2:] != ".t"]
#22s

In [ ]:
loss_fn = nn.CrossEntropyLoss(reduction="sum")
for (name, param) in model.named_parameters():
    if name[-2:] != ".t":
        param.grad = None
        param.requires_grad = True
output = model(input)[..., 0]
loss = loss_fn(output, labels)
loss.backward()
g2 = [param.grad for (name, param) in model.named_parameters() if name[-2:] != ".t"]
#44 s

In [111]:
[(g1[i]-g2[i]).abs().max() for i in range(len(g1))]

[tensor(0.0727),
 tensor(0.0740),
 tensor(0.2588),
 tensor(0.1233),
 tensor(0.2839),
 tensor(0.1607),
 tensor(0.4946),
 tensor(0.1279),
 tensor(2.1909),
 tensor(0.9316)]

In [97]:
g2[4]

tensor([[[[-0.0273, -0.0084],
          [ 0.0100, -0.0439]],

         [[ 0.0052,  0.0143],
          [ 0.0171,  0.0605]],

         [[ 0.0052,  0.0006],
          [-0.0043, -0.0206]],

         ...,

         [[ 0.0026,  0.0006],
          [-0.0454,  0.0176]],

         [[-0.0341, -0.0173],
          [-0.0081, -0.0350]],

         [[ 0.0069,  0.0065],
          [-0.0025,  0.0193]]],


        [[[-0.0085,  0.0151],
          [ 0.0062, -0.0067]],

         [[ 0.0296,  0.0135],
          [-0.0185, -0.0403]],

         [[ 0.0217, -0.0074],
          [-0.0008, -0.0353]],

         ...,

         [[-0.0005, -0.0125],
          [-0.0185,  0.0064]],

         [[ 0.0251, -0.0073],
          [ 0.0349,  0.0368]],

         [[ 0.0155, -0.0091],
          [-0.0155,  0.0016]]],


        [[[-0.0095, -0.0102],
          [-0.0061, -0.0171]],

         [[ 0.0119,  0.0109],
          [ 0.0126,  0.0303]],

         [[ 0.0139,  0.0142],
          [-0.0044,  0.0174]],

         ...,

         [[ 0.0010,  